In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from kserve import (
    constants,
    KServeClient,
    V1beta1InferenceService,
    V1beta1InferenceServiceSpec,
    V1beta1PredictorSpec,
    V1beta1SKLearnSpec,
    V1beta1ModelSpec,
    V1beta1ModelFormat,
    V1beta1TFServingSpec,
    utils
)
from kubernetes import client 

In [3]:
namespace = utils.get_default_target_namespace()
service_name="sklearn-python-student12"

api_version = constants.KSERVE_GROUP + '/' + constants.KSERVE_V1BETA1_VERSION
storage = 'gs://kfserving-examples/models/sklearn/1.0/model'
image = "kserve/sklearnserver:v0.9.0"
isvc = V1beta1InferenceService(api_version=api_version,
                               kind=constants.KSERVE_KIND,
                               metadata=client.V1ObjectMeta(name=service_name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                               spec=V1beta1InferenceServiceSpec(
                                   predictor=V1beta1PredictorSpec(
                                       sklearn=(V1beta1SKLearnSpec(
                                           image = image,
                                            storage_uri = storage))))
)

In [4]:
KServe = KServeClient()
KServe.create(isvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2024-09-25T05:36:56Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {},
        'f:image': {},
        'f:name': {},
        'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2024-09-25T05:36:56Z'}],
  'name': 'sklearn-python-student12',
  'namespace': 'student12-87c78f1c',
  'resourceVersion': '93821865',
  'uid': 'c83d5fe0-5bdd-44b0-bf78-541a23f2f883'},
 'spec': {'predictor': {'model': {'image': 'kserve/sklearnserver:v0.9.0',
    'modelFormat': {'name': 'sklearn'},
    'name': '',
    'resources': {},
    'storageUri': 'gs://kf

In [5]:


KServe.get(service_name, namespace=namespace, watch=True, timeout_seconds=120)
isvc_resp=KServe.get(service_name, namespace=namespace)

NAME                      READY      PREV    LATEST  URL
sklearn-python-student12  Unknown       0       100
NAME                      READY      PREV    LATEST  URL
sklearn-python-student12  Unknown       0       100
NAME                      READY      PREV    LATEST  URL
sklearn-python-student12  Unknown       0       100
NAME                      READY      PREV    LATEST  URL
sklearn-python-student12  Unknown       0       100
NAME                      READY      PREV    LATEST  URL
sklearn-python-student12  True          0       100  https://sklearn-python-student12-predictor.student12-87c78f1c.ezua.ksi.local


In [6]:
import requests
isvc_url = isvc_resp['status']['address']['url']
print(isvc_url)

http://sklearn-python-student12-predictor.student12-87c78f1c.svc.cluster.local


In [7]:
sklearn_iris_input={
    "instances": [
      [6.8,  2.8,  4.8,  1.4],
      [6.0,  3.4,  4.5,  1.6]
    ]
}

In [8]:
ver_info = '/v1/models/'
operaton = ':predict'
rsvc_url = isvc_url + ver_info + service_name + operaton

response = requests.post(rsvc_url, json=sklearn_iris_input)
print(response.text)

{"predictions": [1, 1]}


In [9]:
import requests
import kfp

HOST = "https://kubeflow.ezua.ksi.local/"
USERNAME = "student12@example.com"
PASSWORD = "ezmeral12"

session = requests.Session()
response = session.get(HOST, verify=False) #ssl 접속시 verify=False 

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": USERNAME, "password": PASSWORD}
session.post(response.url, headers=headers, data=data)
#session_cookie = session.cookies.get_dict()["authservice_session"]
session_cookie = session.cookies.get_dict()
print(session_cookie)

SSLError: HTTPSConnectionPool(host='keycloak.ezua.ksi.local', port=443): Max retries exceeded with url: /realms/UA/protocol/openid-connect/auth?client_id=ua&nonce=kWLr08deRywbUeJyTexA32K0UdkNyXASZXKyKxJVNLQ&redirect_uri=https%3A%2F%2Fkubeflow.ezua.ksi.local%2Foauth2%2Fcallback&response_type=code&scope=openid+email+profile+offline_access&state=j5SYmQCrRFVbFjgeTN-lvfBjlqfanitvMvHB-Lw90R4%3Ahttps%3A%2F%2Ftoken-service.ezua.ksi.local%3Forig-url%3Dhttps%3A%2F%2Fkubeflow.ezua.ksi.local%2F (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate (_ssl.c:1006)')))

In [ ]:
import json
#headers = {"Host": "sklearn-iris-python-predictor.demo01.ezua.ksi.local"}
svc_header = "sklearn-python-student99-predictor." + namespace + ".ezua.ksi.local"
headers = {"Host": svc_header }

#res = session.post("https://kubeflow.ezua.ksi.local/v1/models/sklearn-iris-python:predict", headers=headers, cookies=session_cookie, data=json.dumps(sklearn_iris_input))
res = session.post(f"{HOST}v1/models/sklearn-python-student99:predict", headers=headers, cookies=session_cookie, data=json.dumps(sklearn_iris_input))
print(res.json())